This is a draft version to get the MinION reads zipped up a certain way and moved over to a specific HPC for basecalling.

Idea is to point the program towards the reads directory of MinKNOW, zip them up and move them over to a server

In [1]:
#for now name the initial BASEFOLDER, BACKUPFOLDER, NUMBEROFFOLDERS, SEQUENCINGRUN, RTARGETFOLDER
#
#later this will come from the config %%file
BASEFOLDER = '/Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION'
NUMBEROFFOLDERS = 1
SEQUENCINGRUN = 'FLOWID'
BACKUPFOLDER = '/Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION/backup'
RTARGETFOLDER = 'bxs800@r-dm.nci.org.au:/short/xf3/bxs800/test'

In [2]:
import re
import os
import subprocess

In [3]:
#now check if basefolder is there
if not os.path.exists(BASEFOLDER):
    print('Check basefolder')
    exit
if not os.path.exists(BACKUPFOLDER):
    print('Backup folder %s does not exist. Creating it.' %BACKUPFOLDER)
    os.mkdir(BACKUPFOLDER)

In [4]:
#get all the the folders witht the SEQUENCINGRUN ID in the basefolder
_id = re.compile(SEQUENCINGRUN, re.IGNORECASE)

In [5]:
#now get all the folders belonging to this sequencing %run
read_folder_list = [os.path.join(BASEFOLDER, x) for x in os.listdir(BASEFOLDER)]
flowcellfolder = [x for x in read_folder_list if _id.search(x) != None]

In [6]:
#now check if in the target folder is fast5 folder
def check_fast5(x):
    if not os.path.exists(os.path.join(x, 'fast5')):
        return False
    elif not os.path.isdir(os.path.join(x, 'fast5')):
        return False
    else:
        fast5path = os.path.join(x, 'fast5')
        #now check if the first folder contains fast5 files
        subfolders = [os.path.join(fast5path, x) for x in os.listdir(fast5path)]
        if len(subfolders) == 0:
            return False
        else:
            content = [x for x in os.listdir(subfolders[0]) if x.endswith('.fast5')]
            if len(content) == 0:
                return False
            else:
                print(content)
                return True

In [13]:
def tar_zip(fast5_folder,backupfolder, start, end):
    command_list = []
    tar_command = 'tar -czf'
    command_list.append(tar_command)
    folder = fast5_folder.split('/')[-2]
    target = os.path.join(backupfolder,'%s_%s_%s.tar.gz' % (folder, start, end))
    command_list.append(target)
    tar_input = '%s/{%s..%s}'%(fast5_folder, start, end)
    command_list.append(tar_input)
    comand = ' '.join(command_list)
    print(comand)
    #here add in the subprocess
    return comand

In [14]:
def zip_and_backup(x, number, backupfolder):
    fast5path = os.path.join(x, 'fast5')
    #get the runs folder id
    folder = x.split('/')[-1]
    subfolders = [os.path.join(fast5path, x) for x in os.listdir(fast5path)]
    #noffolders = len(subfolders)
    noffolders = 300
    print("We have %i of folders to zip up." % noffolders) 
    if noffolders <= number:
        start = 0
        end = noffolders - 1
        tar_zip(fast5path,backupfolder, start, end)
    elif noffolders > number:
        if number == 1:
            starts = [x for x in range(0, noffolders, number)]
            ends = starts
            start_end_zip = zip(starts, ends)
            for x in start_end_zip:
                start, end = x
                tar_zip(fast5path,backupfolder, start, end)
        else:
            start = 0
            while start < noffolders -number:
                end = start + number -1
                tar_zip(fast5path,backupfolder, start, end)
                start = start + number
            start = end + 1
            end = noffolders -1
            tar_zip(fast5path,backupfolder, start, end)

        

In [15]:
#now zip up all the folders
for x in flowcellfolder:
    print(check_fast5(x))
    zip_and_backup(x, 120 ,  BACKUPFOLDER)

False
We have 300 of folders to zip up.
tar -czf /Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION/backup/date_number_NAME_FLOWID_0_119.tar.gz /Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION/date_number_NAME_FLOWID/fast5/{0..119}
tar -czf /Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION/backup/date_number_NAME_FLOWID_120_239.tar.gz /Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION/date_number_NAME_FLOWID/fast5/{120..239}
tar -czf /Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION/backup/date_number_NAME_FLOWID_240_299.tar.gz /Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION/date_number_NAME_FLOWID/fast5/{240..299}


In [26]:
def scp_zip(file, destination):
    command_list = []
    command_list.append('scp ')
    command_list.append('%s '%file)
    command_list.append('%s'%destination)
    command = ' '.join(command_list)
    #here add subprocess
    print(command)

In [27]:
for file in [os.path.join(BACKUPFOLDER, x) for x in BACKUPFOLDER if x.endswith('tar.gz')]:
    print(file)
    scp_zip(file, RTARGETFOLDER)

In [28]:
for file in [os.path.join(BACKUPFOLDER, x) for x in os.listdir(BACKUPFOLDER) if x.endswith('tar.gz')]:
    print(file)
    scp_zip(file, RTARGETFOLDER)

/Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION/backup/agae.tar.gz
scp  /Users/ben/Documents/work_related/Canberra/Python_and_R_scripts/MinION/backup/agae.tar.gz  bxs800@r-dm.nci.org.au:/short/xf3/bxs800/test
